### 1. Pivot Sales by Year
**Functional Specification:**
* Select `CustomerID`, Year of OrderDate, and `TotalDue` from `Sales.SalesOrderHeader`.
* Use the `PIVOT` operator to transform the Years (2011, 2012, 2013, 2014) into columns.
* Aggregate using `SUM(TotalDue)`.
* Output: CustomerID, [2011], [2012], [2013], [2014].

In [ ]:
Use AdventureWorks2017;
SELECT 
    CustomerID,
    [2011],
    [2012],
    [2013],
    [2014]
FROM (
    SELECT 
        CustomerID,
        YEAR(OrderDate) AS OrderYear,
        TotalDue
    FROM Sales.SalesOrderHeader
) AS SourceData
PIVOT (
    SUM(TotalDue)
    FOR OrderYear IN ([2011], [2012], [2013], [2014])
) AS PivotTable
ORDER BY CustomerID;

### 2. Pivot Employee Counts by Department
**Functional Specification:**
* Join `HumanResources.Employee`, `HumanResources.EmployeeDepartmentHistory`, and `HumanResources.Department`.
* Select `Gender` and `Department Name`.
* Pivot `Department Name` into columns.
* Aggregate using `COUNT(BusinessEntityID)` to see the gender breakdown per department.

In [ ]:
 SELECT 
    p.FirstName + ' ' + p.LastName AS SalesPersonName,
    st.Name AS TerritoryName,
    st.[Group] AS TerritoryGroup,
    sp.TerritoryID,
    FORMAT(sp.SalesYTD, 'C', 'en-US') AS SalesYTD_Formatted,
    sp.SalesYTD,
    ROW_NUMBER() OVER (
        PARTITION BY sp.TerritoryID 
        ORDER BY sp.SalesYTD DESC
    ) AS TerritoryRank
FROM 
    Sales.SalesPerson sp
INNER JOIN 
    Person.Person p ON sp.BusinessEntityID = p.BusinessEntityID
INNER JOIN
    Sales.SalesTerritory st ON sp.TerritoryID = st.TerritoryID
WHERE 
    sp.TerritoryID IS NOT NULL
ORDER BY 
    st.Name, 
    TerritoryRank;


### 3. Pivot Sales by Territory Group
**Functional Specification:**
* Join `Sales.SalesOrderHeader` and `Sales.SalesTerritory`.
* Select Year and `Group` (Europe, North America, Pacific).
* Pivot `Group` into columns.
* Sum `TotalDue`.
* Show sales per year broken down by Territory Group.

In [ ]:
SELECT 
    OrderYear,
    [Europe],
    [North America],
    [Pacific]
FROM (
    SELECT 
        YEAR(soh.OrderDate) AS OrderYear,
        st.[Group] AS TerritoryGroup,
        soh.TotalDue
    FROM Sales.SalesOrderHeader soh
    INNER JOIN Sales.SalesTerritory st
        ON soh.TerritoryID = st.TerritoryID
) AS SourceData
PIVOT (
    SUM(TotalDue)
    FOR TerritoryGroup IN ([Europe], [North America], [Pacific])
) AS PivotTable
ORDER BY OrderYear;

### 4. Pivot Monthly Sales
**Functional Specification:**
* Extract Month (1-12) and Year from `Sales.SalesOrderHeader`.
* Pivot Month numbers into columns [1] through [12].
* Sum `TotalDue`.
* Rows should be Years, Columns should be Months.

In [ ]:
SELECT 
    OrderYear,
    [1], [2], [3], [4], [5], [6], 
    [7], [8], [9], [10], [11], [12]
FROM (
    SELECT 
        YEAR(OrderDate) AS OrderYear,
        MONTH(OrderDate) AS OrderMonth,
        TotalDue
    FROM Sales.SalesOrderHeader
) AS SourceData
PIVOT (
    SUM(TotalDue)
    FOR OrderMonth IN ([1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12])
) AS PivotTable
ORDER BY OrderYear;

### 5. Pivot Purchase Orders by Ship Method
**Functional Specification:**
* Query `Purchasing.PurchaseOrderHeader` and `Purchasing.ShipMethod`.
* Select Year and `ShipMethod` name.
* Pivot ShipMethod.
* Count purchase orders.

In [ ]:
SELECT 
    OrderYear,
    [CARGO TRANSPORT 5] AS CargoTransport,
    [OVERNIGHT J-FAST] AS OvernightFast,
    [XRQ - TRUCK GROUND] AS TruckGround,
    [ZY - EXPRESS] AS Express
FROM (
    SELECT 
        YEAR(poh.OrderDate) AS OrderYear,
        sm.Name AS ShipMethodName,
        poh.PurchaseOrderID
    FROM Purchasing.PurchaseOrderHeader poh
    INNER JOIN Purchasing.ShipMethod sm
        ON poh.ShipMethodID = sm.ShipMethodID
) AS SourceData
PIVOT (
    COUNT(PurchaseOrderID)
    FOR ShipMethodName IN (
        [CARGO TRANSPORT 5],
        [OVERNIGHT J-FAST],
        [XRQ - TRUCK GROUND],
        [ZY - EXPRESS]
    )
) AS PivotTable
ORDER BY OrderYear;

### 6. Unpivot Vendor Credit Ratings
**Functional Specification:**
* (Note: Since AdventureWorks data is normalized, we will simulate a wide table first using a CTE/Derived table to demonstrate UNPIVOT).
* Create a derived table with columns `Vendor1_Credit`, `Vendor2_Credit`, `Vendor3_Credit` (simulated).
* Use `UNPIVOT` to transform these columns into rows: `Vendor` and `Rating`.
* This reverses a cross-tab format into a normalized list.

In [ ]:
WITH VendorCreditWide AS (
    SELECT 
        'Q1-2014' AS Period,
        1 AS Vendor1_Credit,
        2 AS Vendor2_Credit,
        1 AS Vendor3_Credit
    UNION ALL
    SELECT 'Q2-2014', 2, 1, 3
    UNION ALL
    SELECT 'Q3-2014', 1, 2, 2
    UNION ALL
    SELECT 'Q4-2014', 2, 3, 1
)

SELECT 
    Period,
    Vendor,
    CreditRating
FROM VendorCreditWide
UNPIVOT (
    CreditRating
    FOR Vendor IN (Vendor1_Credit, Vendor2_Credit, Vendor3_Credit)
) AS UnpivotTable
ORDER BY Period, Vendor;

### 7. Pivot Product Inventory by Shelf
**Functional Specification:**
* Query `Production.ProductInventory`.
* Select `ProductID` and `Shelf`.
* Pivot `Shelf` (A, B, C, D, etc.).
* Sum `Quantity`.
* Shows how much stock of each product is on each shelf.

In [ ]:
SELECT 
    PivotTable.ProductID,
    p.Name AS ProductName,
    [A], [B], [C], [D], [E], [F], [G], [H], [I], [J],
    [K], [L], [M], [N], [P], [R], [S], [T], [W]
FROM (
    SELECT 
        pi.ProductID,
        pi.Shelf,
        pi.Quantity
    FROM Production.ProductInventory pi
) AS SourceData
PIVOT (
    SUM(Quantity)
    FOR Shelf IN (
        [A], [B], [C], [D], [E], [F], [G], [H], [I], [J],
        [K], [L], [M], [N], [P], [R], [S], [T], [W]
    )
) AS PivotTable
INNER JOIN Production.Product p
    ON PivotTable.ProductID = p.ProductID
ORDER BY PivotTable.ProductID;

### 8. Sales by Customer Region (Pivot)
**Functional Specification:**
* Join Sales headers to `SalesTerritory`.
* Select Year and `CountryRegionCode`.
* Pivot `CountryRegionCode` (US, CA, FR, etc.).
* Sum `TotalDue`. 
* Compare sales performance across countries by year.

In [ ]:
SELECT 
    OrderYear,
    [US] AS United_States,
    [CA] AS Canada,
    [FR] AS France,
    [DE] AS Germany,
    [AU] AS Australia,
    [GB] AS United_Kingdom
FROM (
    SELECT 
        YEAR(soh.OrderDate) AS OrderYear,
        st.CountryRegionCode,
        soh.TotalDue
    FROM Sales.SalesOrderHeader soh
    INNER JOIN Sales.SalesTerritory st
        ON soh.TerritoryID = st.TerritoryID
) AS SourceData
PIVOT (
    SUM(TotalDue)
    FOR CountryRegionCode IN ([US], [CA], [FR], [DE], [AU], [GB])
) AS PivotTable
ORDER BY OrderYear;

### 9. Pivot Days to Manufacture by Product Line
**Functional Specification:**
* Query `Production.Product`.
* Select `ProductLine` and `DaysToManufacture`.
* Pivot `ProductLine` (R, M, T, S).
* Avg `DaysToManufacture`.
* Analyze average manufacturing time per line.

In [ ]:
SELECT 
    'Avg Days to Manufacture' AS Metric,
    [R] AS Road,
    [M] AS Mountain,
    [T] AS Touring,
    [S] AS Standard
FROM (
    SELECT 
        ProductLine,
        DaysToManufacture
    FROM Production.Product
    WHERE ProductLine IS NOT NULL  
) AS SourceData
PIVOT (
    AVG(DaysToManufacture)
    FOR ProductLine IN ([R], [M], [T], [S])
) AS PivotTable;

### 10. Pivot Sales Reasons
**Functional Specification:**
* Join `Sales.SalesOrderHeaderSalesReason` and `Sales.SalesReason`.
* Select `SalesReason.Name` and `SalesOrderID`.
* Pivot the Reason Name (e.g., Price, On Promotion).
* Count `SalesOrderID`.
* See which reasons drive the most orders.

In [ ]:
SELECT 
    'Avg Days to Manufacture' AS Metric,
    [R] AS Road,
    [M] AS Mountain,
    [T] AS Touring,
    [S] AS Standard
FROM (
    SELECT 
        ProductLine,
        DaysToManufacture
    FROM Production.Product
    WHERE ProductLine IS NOT NULL  
) AS SourceData
PIVOT (
    AVG(DaysToManufacture)
    FOR ProductLine IN ([R], [M], [T], [S])
) AS PivotTable;

-- 10. Pivot Sales Reasons
SELECT 
    'Order Count' AS Metric,
    *
FROM (
    SELECT 
        sr.Name AS ReasonName,
        sohr.SalesOrderID
    FROM Sales.SalesOrderHeaderSalesReason sohr
    INNER JOIN Sales.SalesReason sr
        ON sohr.SalesReasonID = sr.SalesReasonID
) AS SourceData
PIVOT (
    COUNT(SalesOrderID)
    FOR ReasonName IN (
        [Price],
        [On Promotion],
        [Magazine Advertisement],
        [Television Advertisement],
        [Manufacturer],
        [Review],
        [Demo Event],
        [Sponsorship],
        [Quality],
        [Other]
    )
) AS PivotTable;